# Chapter 2

## Exercise 1

In this exercise we'll create input parsing functions that parse datasets of [Premier League results](https://github.com/footballcsv/england).

In [1]:
import os
import numpy as np
import pandas as pd

## Problem 1

Create a function `load_matches` that does the following:

- It takes a single csv file from the `../data/england-master/XXXXs/XXXX-XX/eng.1.csv`-datasets.
- It reads the csv.
- It converts Date into a proper Datetime object.
- It determines the season of the dataset and stores it into column `Season`. E.g. data files in 2019-20 have season 2019.
- It returns the dataset.

Hint: [min](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.min.html) and [to_period](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#time-span-representation) might be of interest.

In [2]:
def load_matches(datafile):
    matches = pd.read_csv(datafile)
    matches.loc[:,'Date'] = pd.to_datetime(matches.loc[:,'Date'])
    matches['Season'] = matches.loc[:,'Date'].min().to_period('Y')
    return matches

In [3]:
matches_19 = load_matches('../data/england-master/2010s/2019-20/eng.1.csv')
matches_19.head()

,Round,Date,Team 1,FT,Team 2,Season
0,1,2019-08-09,Liverpool FC,4-1,Norwich City FC,2019
1,1,2019-08-10,West Ham United FC,0-5,Manchester City FC,2019
2,1,2019-08-10,Burnley FC,3-0,Southampton FC,2019
3,1,2019-08-10,AFC Bournemouth,1-1,Sheffield United FC,2019
4,1,2019-08-10,Crystal Palace FC,0-0,Everton FC,2019


## Preamble for problem 2

We want to convert our data into a tidy format with the following columns:

- `Round` - same as initial data
- `Date` - same as initial data
- `Team` - new column that tells which team is in question
- `Opponent` - new column that tells who was the opponent for the `Team`
- `Season` - same as initial data
- `ScoredGoals` - new column that tells how many goals the `Team` scored
- `AllowedGoals` - new column that tells how many goals the `Team` allowed
- `Side` - new column that tells whether the `Team` played on the `Home`-side or on the `Away`-side.
- `Result` - new column that tells the result from values `Win`, `Loss` or `Draw`
- `Points` - new column that lists the amount of league points the team received from the game (3 for `Win`, 1 for `Draw`, 0 for `Loss`)

The output of this formatting function should look something like this:

||Round|Date|Team|Opponent|Season|ScoredGoals|AllowedGoals|Side|Result|Points|
|-|-|-|-|-|-|-|-|-|-|-|
|0|1|2019-08-09|Liverpool FC|Norwich City FC|2019|4|1|Home|Win|3|
|1|1|2019-08-10|West Ham United FC|Manchester City FC|2019|0|5|Home|Loss|0|

Do note that in this tidy format we have doubled the amount of rows: the original format had two results, one win and one loss, encoded into the column `FT`.

In our new format we have two differents states: one from both perspectives. This allows us to calculate e.g. `Points`, which are allocated in an asymmetric fashion (if the match wasn't a draw).

## Problem 2

Let's create a function `format_matches`, which takes a single DataFrame created by `load_matches` and converts the data into our desired data format:

- Use [pandas.Series.str.extract](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extract.html) to extract `HomeGoals` and `AwayGoals` from the `FT`-column. Convert these columns into integers.
- Drop unneeded `FT`-column.
- Create two copies of the initial dataset: one from the away perspective and one from the home team perspective.
- For home side:
    - Set `Side` to `Home`.
    - Rename `Team 1` to `Team`, `Team 2` to `Opponent`, `Homegoals` to `ScoredGoals` and `AwayGoals` to `AllowedGoals`.
    - Set `Result` to `Win` if home team one, `Draw`, if match was a draw and `Loss` if away team won.
- For away side:
    - Set `Side` to `Away`.
    - Rename `Team 1` to `Opponent`, `Team 2` to `Team`, `Homegoals` to `AllowedGoals` and `AwayGoals` to `ScoredGoals`.
    - Set `Result` to `Loss` if home team one, `Draw`, if match was a draw and `Win` if away team won.
- Concatenate both home- and away-datasets together to get all matches.
- Create column `Points` and fill it based on the `Result`-column. 3 points for a `Win`, 1 point for a `Draw` and 0 points for a `Loss`.

Hint: You can use [boolean indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#boolean-indexing) to test which team won based on `HomeGoals` and `AwayGoals`.

In [4]:
def format_matches(matches):
    matches.loc[:, ['HomeGoals','AwayGoals']] = matches.loc[:, 'FT'].str.extract('(?P<HomeGoals>\d)-(?P<AwayGoals>\d)').astype(np.int64)
    matches.drop('FT', axis=1, inplace=True)
    home_win = matches['HomeGoals'] > matches['AwayGoals']
    draw = matches['HomeGoals'] == matches['AwayGoals']
    
    home_matches = matches.copy()
    home_matches['Side'] = 'Home'
    home_matches = home_matches.rename(columns={'Team 1':'Team', 'Team 2': 'Opponent', 'HomeGoals': 'ScoredGoals', 'AwayGoals': 'AllowedGoals'})
    home_matches.loc[home_win,'Result'] = 'Win'
    home_matches.loc[draw,'Result'] = 'Draw'
    home_matches.loc[-(draw|home_win),'Result'] = 'Loss'
    
    away_matches = matches.copy()
    away_matches['Side'] = 'Away'
    away_matches = away_matches.rename(columns={'Team 1':'Opponent', 'Team 2': 'Team', 'HomeGoals': 'AllowedGoals', 'AwayGoals': 'ScoredGoals'})
    away_matches.loc[home_win,'Result'] = 'Loss'
    away_matches.loc[draw,'Result'] = 'Draw'
    away_matches.loc[-(draw|home_win),'Result'] = 'Win'
    
    all_matches = pd.concat([home_matches,away_matches])
    all_matches['Points'] = 0
    all_matches.loc[all_matches.loc[:,'Result'] == 'Win','Points'] = 3
    all_matches.loc[all_matches.loc[:,'Result'] == 'Draw','Points'] = 1
    return all_matches

In [5]:
matches_19 = format_matches(matches_19)
matches_19.head()

,Round,Date,Team,Opponent,Season,ScoredGoals,AllowedGoals,Side,Result,Points
0,1,2019-08-09,Liverpool FC,Norwich City FC,2019,4,1,Home,Win,3
1,1,2019-08-10,West Ham United FC,Manchester City FC,2019,0,5,Home,Loss,0
2,1,2019-08-10,Burnley FC,Southampton FC,2019,3,0,Home,Win,3
3,1,2019-08-10,AFC Bournemouth,Sheffield United FC,2019,1,1,Home,Draw,1
4,1,2019-08-10,Crystal Palace FC,Everton FC,2019,0,0,Home,Draw,1


## Problem 3

- Create a function `clean_matches`, which converts `Team`-, `Opponent`- and `Result`-categories from a dataset produced by our `format_matches` into categorical datatype.
- Think why this step needs to be a separate function. Why couldn't these steps be part of the `format_matches`-function?

In [6]:
def clean_matches(matches):
    matches.loc[:,['Team', 'Opponent', 'Result']] = matches.loc[:, ['Team', 'Opponent', 'Result']].astype('category')
    return matches

In [7]:
matches_19 = clean_matches(matches_19)
matches_19.dtypes

Round                    int64
Date            datetime64[ns]
Team                  category
Opponent              category
Season           period[A-DEC]
ScoredGoals              int64
AllowedGoals             int64
Side                    object
Result                category
Points                   int64
dtype: object

## Problem 4

Create a function `read_matches` which, given the directory `../data/england-master/` and a list of seasons, does the following:

- Iterates over the given seasons
- Determines the correct data file based on season
- Loads the data using `load_matches` and formats the data using `format_matches`
- Concatenates different datasets together
- Runs `clean_matches` on the combined dataset
- Returns cleaned dataset with multiple seasons

Hint: Python [string formatting](https://docs.python.org/3/library/string.html#format-specification-mini-language) might be of interest.

In [8]:
def read_matches(matchfolder, seasons):
    datasets = []

    for season in seasons:
        decadestr = '%d0s' % int(season/10)
        seasonstr = '%d-%02d' % (season, (int(str(season)[-2:])+1)%100)
        
        datapath = os.path.join(matchfolder, decadestr, seasonstr,'eng.1.csv')
        datasets.append(format_matches(load_matches(datapath)))

    match_data = clean_matches(pd.concat(datasets))
    return match_data

In [9]:
matches_all = read_matches('../data/england-master/', range(1992,2020))

## Demonstration of our new data format

### Checking home side advantage

**Requirement**: Problems 1 and 2 need to be completed for the dataset

Let's use our new datasets to check whether the home side has an advantage compared to the away side. This phenomenon [has been recognized](https://www.researchgate.net/publication/14465849_Factors_associated_with_home_advantage_in_English_and_Scottish_Soccer_matches) for years. Let's see if our data shows the same phenomena.

For this let's use a [binomial test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binom_test.html) to check whether our wins and draws come from a fair binomial distribution (wins and losses as likely).

In [10]:
from scipy.stats import binom_test
def home_advantage(matches):
    home_team_stats = matches.loc[(matches.loc[:,'Side'] == 'Home'),'Result'].value_counts()
    print(home_team_stats)
    print('Home side wins %.1f %% of matches that do not end in a draw' % float(100.0*home_team_stats.loc['Win']/(home_team_stats.loc['Win']+home_team_stats.loc['Loss'])))
    return binom_test(home_team_stats.loc[['Win','Loss']])

In [11]:
home_advantage(matches_19)
home_advantage(matches_all)

Win     172
Loss    116
Draw     92
Name: Result, dtype: int64
Home side wins 59.7 % of matches that do not end in a draw
Win     5028
Loss    3019
Draw    2839
Name: Result, dtype: int64
Home side wins 62.5 % of matches that do not end in a draw


3.8740473705693534e-112

### Checking Premier League winners

**Requirement**: Problems 1 and 2 need to be completed for the dataset

Let's use our new datasets to check which teams won seasons by checking which team got the most points on each season.

We can then compare teams with the most points to this [list of Premier League champions](https://en.wikipedia.org/wiki/List_of_English_football_champions#Premier_League_(1992%E2%80%93present)) and see that the lists match.

In [12]:
def season_winners(matches):
    team_standings = matches.loc[:,['Season','Team','Points']].groupby(['Season','Team']).sum()
    team_standings.reset_index(inplace=True)
    team_standings.dropna(inplace=True)
    top_teams = team_standings.groupby('Season').apply(lambda x: x.nlargest(1, 'Points'))
    return top_teams

In [13]:
season_winners(matches_all)

,,Season,Team,Points
Season,,,,
1992,26,1992,Manchester United FC,84.0
1993,75,1993,Manchester United FC,92.0
1994,103,1994,Blackburn Rovers FC,89.0
1995,173,1995,Manchester United FC,82.0
1996,222,1996,Manchester United FC,75.0
1997,246,1997,Arsenal FC,78.0
1998,320,1998,Manchester United FC,79.0
1999,369,1999,Manchester United FC,91.0
2000,418,2000,Manchester United FC,80.0
